In [170]:
import pandas as pd
import sys
sys.path.append('/Users/halukamb/Documents/CodeRelated/HBDataviz_new/2025/01_BTW100Deutsche/Wrangle/lib/python3.12/site-packages')
import numpy as np
from json import loads, dumps

In [2]:
! pip install pyreadstat

In [3]:
def germanElectorate(df):
    german_df=df[df["german"]=="JA"]
    german_df=german_df.dropna(subset="age")
    german_df["age"]=german_df["age"].astype(float)
    german_electorate=german_df[german_df["age"]>=18]
    return german_electorate

In [136]:
def reformgroups(df):
    df["Alter"]=None
    df["Alter"]=np.where(df["age"]<30,"U30",df["Alter"])
    df["Alter"]=np.where(df["age"]>=30,"30-49",df["Alter"])
    df["Alter"]=np.where(df["age"]>=50,"50-64",df["Alter"])
    df["Alter"]=np.where(df["age"]>64,"Ü65",df["Alter"])
    df["Geschlecht"]=np.nan
    df["Geschlecht"]=np.where(df["sex"]=="FRAU","Weiblich",df["Geschlecht"])
    df["Geschlecht"]=np.where(df["sex"]=="MANN","Männlich",df["Geschlecht"])
    df["OstWest"]=np.nan
    df["OstWest"]=np.where(df["eastwest"]=="ALTE BUNDESLAENDER","West",df["OstWest"])
    df["OstWest"]=np.where(df["eastwest"]=="NEUE BUNDESLAENDER","Ost",df["OstWest"])
    df["OstWest"]=np.where(df["land"].str.contains("BERLIN"),"Berlin",df["OstWest"])
    df["Besiedelung"]=np.nan
    df["Besiedelung"]=np.where(df["gs01"].isin(['GROSSSTADT']),"dicht besiedelt",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['MITTEL-, KLEINSTADT', 'VORORT GROSSSTADT']),"mittlere Besiedlungsdichte",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['LAENDL. DORF','EINZELHAUS, LAND']),"gering besiedelt",df["Besiedelung"])
    
    


    
    return df

In [137]:
df_2018 = pd.read_spss("../data/AllbusData/ZA5270_v2-0-0.sav")
df_2018 = germanElectorate(df_2018)
df_2018= reformgroups(df_2018)
df_2018.Alter.unique()

array(['50-64', 'U30', '30-49', 'Ü65'], dtype=object)

In [138]:
df_2021 = pd.read_spss("../data/AllbusData/ZA5280_v2-0-1.sav")
df_2021 = germanElectorate(df_2021)
df_2021= reformgroups(df_2021)

In [140]:
df_2023 = pd.read_spss("../data/AllbusData/ZA8830_v1-0-0.sav")
df_2023 = germanElectorate(df_2023)
df_2023= reformgroups(df_2023)


In [141]:
for i in df_2023["land"].unique():
    print(i)

RHEINLAND-PFALZ
BAYERN
THUERINGEN
SCHLESWIG-HOLSTEIN
BADEN-WUERTTEMBERG
NIEDERSACHSEN
BRANDENBURG
NORDRHEIN-WESTFALEN
SACHSEN
HESSEN
EHEM. BERLIN-OST
EHEM. BERLIN-WEST
MECKLENB.-VORPOMMERN
SACHSEN-ANHALT
HAMBURG
SAARLAND
BREMEN


In [142]:
factor_list=["Alter","Geschlecht","OstWest","Besiedelung"]

In [143]:
def analyze_factor_patterns(dataset,variable_to_check):
    base_df=pd.DataFrame()
    for element in factor_list:
        #print(element)
        for element2 in factor_list:
            #print(element2)
            baselist=[variable_to_check,"za_nr"]
            extralist=[element,element2]
            extralist=list(set(extralist))
    
            baselist.extend(extralist)
            dataset_reduced=dataset[baselist]
            dataset_reduced = dataset_reduced[~dataset_reduced.isin(["nan"]).any(axis=1)]
            if len(dataset_reduced)<50:
                print("low")
            extralist_plus_factor=extralist+[variable_to_check]
            grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
            grouped_df=grouped_df.dropna(axis=1)
            grouped_df['percent'] = grouped_df['za_nr']*100 / grouped_df.groupby(by=extralist)['za_nr'].transform('sum')

            
            grouped_df=grouped_df.reset_index()
            base_df=pd.concat([base_df,grouped_df],axis=0)
    base_df=base_df.drop_duplicates()
    base_df_cols=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', variable_to_check,
       'za_nr','percent']
    
    return base_df[base_df_cols]

In [144]:
df_2023_inflation=analyze_factor_patterns(df_2023,"va03")
df_2023_inflation

/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_16106/3226200612.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_16106/3226200612.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_16106/3226200612.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,va03,za_nr,percent
0,30-49,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,342,0.266771
1,30-49,NaN,NaN,NaN,AM VIERTWICHTIGSTEN,425,0.331513
2,30-49,NaN,NaN,NaN,AM WICHTIGSTEN,213,0.166147
3,30-49,NaN,NaN,NaN,AM ZWEITWICHTIGSTEN,302,0.235569
4,50-64,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,396,0.289898
...,...,...,...,...,...,...,...
7,NaN,gering besiedelt,NaN,NaN,AM ZWEITWICHTIGSTEN,292,0.215817
8,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM DRITTWICHTIGSTEN,613,0.273661
9,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,895,0.399554
10,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,260,0.116071


In [145]:
df_2021_inflation=analyze_factor_patterns(df_2021,"va03")
df_2021_inflation

/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_16106/3226200612.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_16106/3226200612.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_16106/3226200612.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,va03,za_nr,percent
0,30-49,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,223,0.261430
1,30-49,NaN,NaN,NaN,AM VIERTWICHTIGSTEN,405,0.474795
2,30-49,NaN,NaN,NaN,AM WICHTIGSTEN,74,0.086753
3,30-49,NaN,NaN,NaN,AM ZWEITWICHTIGSTEN,151,0.177022
4,50-64,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,242,0.240079
...,...,...,...,...,...,...,...
7,NaN,gering besiedelt,NaN,NaN,AM ZWEITWICHTIGSTEN,163,0.169792
8,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM DRITTWICHTIGSTEN,376,0.250667
9,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,837,0.558000
10,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,66,0.044000


In [146]:
comp_inflation=pd.merge(df_2021_inflation,df_2023_inflation,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'va03'],suffixes=["_2021","_2023"])
comp_inflation

,Alter,Besiedelung,Geschlecht,OstWest,va03,za_nr_2021,percent_2021,za_nr_2023,percent_2023
0,30-49,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,223,0.261430,342,0.266771
1,30-49,NaN,NaN,NaN,AM VIERTWICHTIGSTEN,405,0.474795,425,0.331513
2,30-49,NaN,NaN,NaN,AM WICHTIGSTEN,74,0.086753,213,0.166147
3,30-49,NaN,NaN,NaN,AM ZWEITWICHTIGSTEN,151,0.177022,302,0.235569
4,50-64,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,242,0.240079,396,0.289898
...,...,...,...,...,...,...,...,...,...
255,NaN,gering besiedelt,NaN,NaN,AM ZWEITWICHTIGSTEN,163,0.169792,292,0.215817
256,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM DRITTWICHTIGSTEN,376,0.250667,613,0.273661
257,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,837,0.558000,895,0.399554
258,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,66,0.044000,260,0.116071


In [147]:
df_2021_party=analyze_factor_patterns(df_2021,"pv01")
df_2021_party

/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_16106/3226200612.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_16106/3226200612.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_16106/3226200612.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,pv01,za_nr,percent
0,30-49,NaN,NaN,NaN,AFD,75,0.073242
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,62,0.060547
2,30-49,NaN,NaN,NaN,CDU-CSU,215,0.209961
3,30-49,NaN,NaN,NaN,DIE GRUENEN,306,0.298828
4,30-49,NaN,NaN,NaN,DIE LINKE,69,0.067383
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,432,0.230523
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,145,0.077375
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,230,0.122732
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,296,0.157951


In [148]:
df_2023_party=analyze_factor_patterns(df_2023,"pv01")
df_2023_party

/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_16106/3226200612.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_16106/3226200612.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_16106/3226200612.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,pv01,za_nr,percent
0,30-49,NaN,NaN,NaN,AFD,160,0.157171
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,47,0.046169
2,30-49,NaN,NaN,NaN,CDU-CSU,206,0.202358
3,30-49,NaN,NaN,NaN,DIE GRUENEN,214,0.210216
4,30-49,NaN,NaN,NaN,DIE LINKE,67,0.065815
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,322,0.173491
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,115,0.061961
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,162,0.087284
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,371,0.199892


In [156]:
comp_inflation["diff_percent"]=comp_inflation["percent_2023"]-comp_inflation["percent_2021"]
comp_inflation[comp_inflation["va03"]=="AM WICHTIGSTEN"].sort_values(by="diff_percent",ascending=False)
#comp_inflation

,Alter,Besiedelung,Geschlecht,OstWest,va03,za_nr_2021,percent_2021,za_nr_2023,percent_2023,diff_percent
106,U30,dicht besiedelt,NaN,NaN,AM WICHTIGSTEN,7,0.064815,43,0.223958,0.159144
90,U30,NaN,NaN,West,AM WICHTIGSTEN,18,0.068702,89,0.213942,0.145240
26,U30,NaN,Männlich,NaN,AM WICHTIGSTEN,13,0.074286,58,0.199313,0.125027
138,U30,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,11,0.082090,50,0.204918,0.122828
42,U30,NaN,Weiblich,NaN,AM WICHTIGSTEN,19,0.106742,64,0.229391,0.122649
...,...,...,...,...,...,...,...,...,...,...
70,50-64,NaN,NaN,Ost,AM WICHTIGSTEN,13,0.041009,26,0.062350,0.021341
118,50-64,gering besiedelt,NaN,NaN,AM WICHTIGSTEN,15,0.042493,29,0.062099,0.019606
54,50-64,NaN,NaN,Berlin,AM WICHTIGSTEN,2,0.040816,3,0.053571,0.012755
110,Ü65,dicht besiedelt,NaN,NaN,AM WICHTIGSTEN,6,0.040541,11,0.044715,0.004175


In [157]:
comp_party=pd.merge(df_2021_party,df_2023_party,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'pv01'],suffixes=["_2021","_2023"])
comp_party["diff_percent"]=comp_party["percent_2023"]-comp_party["percent_2021"]
comp_party.sort_values("diff_percent")
comp_party_pivot=comp_party.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='pv01')[['percent_2021','percent_2023']].reset_index()

new_columns=[]
for index in comp_party_pivot:
    if index[1]!="":
        new_columns.append(index[1]+"_"+index[0])
    else:
        new_columns.append(index[0])
new_columns
comp_party_pivot.columns=new_columns
comp_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,DIE LINKE_percent_2021,FDP_percent_2021,SPD_percent_2021,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023
0,NaN,NaN,NaN,Berlin,0.056818,0.039773,0.193182,0.284091,0.096591,0.051136,0.204545,0.073864,0.097938,0.041237,0.154639,0.231959,0.201031,0.046392,0.180412,0.046392
1,NaN,NaN,NaN,Ost,0.116905,0.052986,0.238015,0.142977,0.132044,0.115223,0.136249,0.065601,0.211712,0.043243,0.209009,0.105405,0.116216,0.059459,0.193694,0.061261
2,NaN,NaN,NaN,West,0.045792,0.035479,0.276815,0.273102,0.048680,0.124587,0.152228,0.043317,0.082716,0.040741,0.275720,0.224691,0.031276,0.094650,0.200823,0.049383
3,NaN,NaN,Männlich,NaN,0.093082,0.039166,0.255849,0.203459,0.079858,0.131231,0.150051,0.047304,0.145246,0.042320,0.254963,0.166144,0.057994,0.092476,0.188088,0.052769
4,NaN,NaN,Männlich,Berlin,0.087912,0.032967,0.153846,0.263736,0.087912,0.076923,0.219780,0.076923,0.111111,0.055556,0.166667,0.211111,0.188889,0.044444,0.177778,0.044444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,0.084530,0.012759,0.318979,0.132376,0.078150,0.116427,0.216906,0.039872,0.100334,0.011706,0.342809,0.103679,0.060201,0.068562,0.270903,0.041806
61,Ü65,NaN,Weiblich,NaN,0.022263,0.022263,0.369202,0.152134,0.085343,0.100186,0.202226,0.046382,0.065603,0.024823,0.308511,0.145390,0.054965,0.062057,0.290780,0.047872
62,Ü65,dicht besiedelt,NaN,NaN,0.067416,0.016854,0.308989,0.151685,0.056180,0.084270,0.258427,0.056180,0.074766,0.018692,0.233645,0.107477,0.079439,0.079439,0.364486,0.042056
63,Ü65,gering besiedelt,NaN,NaN,0.063291,0.018987,0.335443,0.132911,0.079114,0.107595,0.202532,0.060127,0.076190,0.025397,0.393651,0.130159,0.031746,0.060317,0.244444,0.038095


In [158]:
comp_inflation_pivot=comp_inflation.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='va03')[['percent_2021','percent_2023']].reset_index()
new_columns=[]
for index in comp_inflation_pivot:
    if index[1]!="":
        new_columns.append(index[1]+"_"+index[0])
    else:
        new_columns.append(index[0])
new_columns
comp_inflation_pivot.columns=new_columns
comp_inflation_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,AM WICHTIGSTEN_percent_2023,AM ZWEITWICHTIGSTEN_percent_2023
0,NaN,NaN,NaN,Berlin,0.254777,0.477707,0.070064,0.197452,0.295154,0.356828,0.110132,0.237885
1,NaN,NaN,NaN,Ost,0.259740,0.510490,0.062937,0.166833,0.260838,0.408960,0.115607,0.214595
2,NaN,NaN,NaN,West,0.248855,0.562850,0.045802,0.142494,0.284419,0.393553,0.115178,0.206850
3,NaN,NaN,Männlich,NaN,0.249841,0.552315,0.050095,0.147749,0.294563,0.418449,0.098485,0.188503
4,NaN,NaN,Männlich,Berlin,0.197183,0.577465,0.056338,0.169014,0.339623,0.424528,0.084906,0.150943
...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,0.244813,0.578838,0.024896,0.151452,0.282511,0.469357,0.058296,0.189836
61,Ü65,NaN,Weiblich,NaN,0.247642,0.573113,0.021226,0.158019,0.258297,0.448773,0.077922,0.215007
62,Ü65,dicht besiedelt,NaN,NaN,0.263514,0.527027,0.040541,0.168919,0.256098,0.508130,0.044715,0.191057
63,Ü65,gering besiedelt,NaN,NaN,0.241667,0.608333,0.016667,0.133333,0.269841,0.441799,0.079365,0.208995


In [159]:
comp_inflation_party_pivot=pd.merge(comp_inflation_pivot,comp_party_pivot,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])
comp_inflation_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,SPD_percent_2021,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023
0,NaN,NaN,NaN,Berlin,0.254777,0.477707,0.070064,0.197452,0.295154,0.356828,...,0.204545,0.073864,0.097938,0.041237,0.154639,0.231959,0.201031,0.046392,0.180412,0.046392
1,NaN,NaN,NaN,Ost,0.259740,0.510490,0.062937,0.166833,0.260838,0.408960,...,0.136249,0.065601,0.211712,0.043243,0.209009,0.105405,0.116216,0.059459,0.193694,0.061261
2,NaN,NaN,NaN,West,0.248855,0.562850,0.045802,0.142494,0.284419,0.393553,...,0.152228,0.043317,0.082716,0.040741,0.275720,0.224691,0.031276,0.094650,0.200823,0.049383
3,NaN,NaN,Männlich,NaN,0.249841,0.552315,0.050095,0.147749,0.294563,0.418449,...,0.150051,0.047304,0.145246,0.042320,0.254963,0.166144,0.057994,0.092476,0.188088,0.052769
4,NaN,NaN,Männlich,Berlin,0.197183,0.577465,0.056338,0.169014,0.339623,0.424528,...,0.219780,0.076923,0.111111,0.055556,0.166667,0.211111,0.188889,0.044444,0.177778,0.044444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,0.244813,0.578838,0.024896,0.151452,0.282511,0.469357,...,0.216906,0.039872,0.100334,0.011706,0.342809,0.103679,0.060201,0.068562,0.270903,0.041806
61,Ü65,NaN,Weiblich,NaN,0.247642,0.573113,0.021226,0.158019,0.258297,0.448773,...,0.202226,0.046382,0.065603,0.024823,0.308511,0.145390,0.054965,0.062057,0.290780,0.047872
62,Ü65,dicht besiedelt,NaN,NaN,0.263514,0.527027,0.040541,0.168919,0.256098,0.508130,...,0.258427,0.056180,0.074766,0.018692,0.233645,0.107477,0.079439,0.079439,0.364486,0.042056
63,Ü65,gering besiedelt,NaN,NaN,0.241667,0.608333,0.016667,0.133333,0.269841,0.441799,...,0.202532,0.060127,0.076190,0.025397,0.393651,0.130159,0.031746,0.060317,0.244444,0.038095


In [160]:
zensus=pd.read_csv("json_none.csv")
zensus

,Geschlecht,Besiedelung,OstWest,Alter,Share
0,NaN,NaN,Berlin,U30,0.662670
1,NaN,NaN,Berlin,50-64,1.008255
2,NaN,NaN,Berlin,Ü65,1.054885
3,NaN,NaN,Berlin,30-49,1.264092
4,Männlich,NaN,Berlin,NaN,1.910200
...,...,...,...,...,...
58,Weiblich,NaN,West,NaN,41.055375
59,NaN,mittlere Besiedlungsdichte,NaN,NaN,44.163973
60,Männlich,NaN,NaN,NaN,48.323387
61,Weiblich,NaN,NaN,NaN,51.619587


In [172]:
comp_inflation_party_pivot_zensus=pd.merge(comp_inflation_party_pivot,zensus,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])
comp_inflation_party_pivot_zensus

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,NaN,NaN,NaN,Berlin,0.254777,0.477707,0.070064,0.197452,0.295154,0.356828,...,0.073864,0.097938,0.041237,0.154639,0.231959,0.201031,0.046392,0.180412,0.046392,3.989901
1,NaN,NaN,NaN,Ost,0.259740,0.510490,0.062937,0.166833,0.260838,0.408960,...,0.065601,0.211712,0.043243,0.209009,0.105405,0.116216,0.059459,0.193694,0.061261,16.433332
2,NaN,NaN,NaN,West,0.248855,0.562850,0.045802,0.142494,0.284419,0.393553,...,0.043317,0.082716,0.040741,0.275720,0.224691,0.031276,0.094650,0.200823,0.049383,79.519741
3,NaN,NaN,Männlich,NaN,0.249841,0.552315,0.050095,0.147749,0.294563,0.418449,...,0.047304,0.145246,0.042320,0.254963,0.166144,0.057994,0.092476,0.188088,0.052769,48.323387
4,NaN,NaN,Männlich,Berlin,0.197183,0.577465,0.056338,0.169014,0.339623,0.424528,...,0.076923,0.111111,0.055556,0.166667,0.211111,0.188889,0.044444,0.177778,0.044444,1.910200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Ü65,NaN,Männlich,NaN,0.244813,0.578838,0.024896,0.151452,0.282511,0.469357,...,0.039872,0.100334,0.011706,0.342809,0.103679,0.060201,0.068562,0.270903,0.041806,12.463294
59,Ü65,NaN,Weiblich,NaN,0.247642,0.573113,0.021226,0.158019,0.258297,0.448773,...,0.046382,0.065603,0.024823,0.308511,0.145390,0.054965,0.062057,0.290780,0.047872,15.997430
60,Ü65,dicht besiedelt,NaN,NaN,0.263514,0.527027,0.040541,0.168919,0.256098,0.508130,...,0.056180,0.074766,0.018692,0.233645,0.107477,0.079439,0.079439,0.364486,0.042056,9.247415
61,Ü65,gering besiedelt,NaN,NaN,0.241667,0.608333,0.016667,0.133333,0.269841,0.441799,...,0.060127,0.076190,0.025397,0.393651,0.130159,0.031746,0.060317,0.244444,0.038095,6.025729


In [173]:
comp_inflation_party_pivot_zensus

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,...,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,NaN,NaN,NaN,Berlin,0.254777,...,0.201031,0.046392,0.180412,0.046392,3.989901
1,NaN,NaN,NaN,Ost,0.259740,...,0.116216,0.059459,0.193694,0.061261,16.433332
2,NaN,NaN,NaN,West,0.248855,...,0.031276,0.094650,0.200823,0.049383,79.519741
3,NaN,NaN,Männlich,NaN,0.249841,...,0.057994,0.092476,0.188088,0.052769,48.323387
4,NaN,NaN,Männlich,Berlin,0.197183,...,0.188889,0.044444,0.177778,0.044444,1.910200
...,...,...,...,...,...,...,...,...,...,...,...
58,Ü65,NaN,Männlich,NaN,0.244813,...,0.060201,0.068562,0.270903,0.041806,12.463294
59,Ü65,NaN,Weiblich,NaN,0.247642,...,0.054965,0.062057,0.290780,0.047872,15.997430
60,Ü65,dicht besiedelt,NaN,NaN,0.263514,...,0.079439,0.079439,0.364486,0.042056,9.247415
61,Ü65,gering besiedelt,NaN,NaN,0.241667,...,0.031746,0.060317,0.244444,0.038095,6.025729


In [171]:
result = comp_inflation_party_pivot_zensus.to_json(orient="records")
parsed = loads(result)
with open('data_for_viz.json', 'w', encoding='utf-8') as f:
    json.dump(parsed, f)

[{'Alter': '30-49',
  'Besiedelung': 'dicht besiedelt',
  'Geschlecht': None,
  'OstWest': None,
  'AM DRITTWICHTIGSTEN_percent_2021': 0.3165829146,
  'AM VIERTWICHTIGSTEN_percent_2021': 0.4572864322,
  'AM WICHTIGSTEN_percent_2021': 0.0753768844,
  'AM ZWEITWICHTIGSTEN_percent_2021': 0.1507537688,
  'AM DRITTWICHTIGSTEN_percent_2023': 0.3076923077,
  'AM VIERTWICHTIGSTEN_percent_2023': 0.3321678322,
  'AM WICHTIGSTEN_percent_2023': 0.1503496503,
  'AM ZWEITWICHTIGSTEN_percent_2023': 0.2097902098,
  'AFD_percent_2021': 0.0357142857,
  'ANDERE PARTEI_percent_2021': 0.0476190476,
  'CDU-CSU_percent_2021': 0.1507936508,
  'DIE GRUENEN_percent_2021': 0.4642857143,
  'DIE LINKE_percent_2021': 0.1031746032,
  'FDP_percent_2021': 0.0873015873,
  'SPD_percent_2021': 0.0714285714,
  'WUERDE NICHT WAEHLEN_percent_2021': 0.0396825397,
  'AFD_percent_2023': 0.0991735537,
  'ANDERE PARTEI_percent_2023': 0.0454545455,
  'CDU-CSU_percent_2023': 0.1074380165,
  'DIE GRUENEN_percent_2023': 0.3347107438